# Basic

In [81]:
import pandas as pd
import time
from tqdm import tqdm
import requests
import json
from urllib import parse

raw_path = "./data/raw/"
temp_path = "./data/temp/"
data_path = "./data/"

In [88]:
def get_location(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?query=" + address
    headers = {"Authorization": "KakaoAK d1a252f9abb2970a266494fd8b7161ad"}
    api_json = json.loads(str(requests.get(url, headers=headers).text))
    address = api_json["documents"][0]["address"]
    return float(address["y"]), float(address["x"]), address["address_name"]

In [161]:
def whether_gangnam(latitude, longitude):    
    url = "https://apis.openapi.sk.com/tmap/geo/reversegeocoding?version=1"

    payload = {
        "appKey": json.load(open("./data/secrets.json", "r"))["tmap"]["appkey"],
        "lon": longitude,
        "lat": latitude,
    }

    res = requests.get(url, params=payload)

    try:
        gu = json.loads(res.text)["addressInfo"]["gu_gun"]
    except:
        return "ERROR"

    if gu == "강남구":
        return True
    else:
        return False

In [28]:
def get_geocoding(dong, bunji):
    url = "https://apis.openapi.sk.com/tmap/geo/reversegeocoding?version=1"
    
    payload = {
        "appKey": json.load(open("./data/secrets.json", "r"))["tmap"]["appkey"],
        "city_do": quote("서울특별시"),
        "gu_gun": quote("강남구"),
        "dong": quote(dong),
        "bunji": quote(bunji),
        "addressFlag": "F02"
    }
    
    res = requests.get(url, params=payload)
    
    try:
        info = json.loads(res.text)["coordinateInfo"]
        latitude = info["lat"]
        longitude = info["lon"]
    except:
        return "ERROR"
    
    return (latitude, longitude)

In [74]:
url = "https://apis.openapi.sk.com/tmap/geo/geocoding?version=1"

headers = {"appKey": json.load(open("./data/secrets.json", "r"))["tmap"]["appkey"]}

payload = {
    "appKey": json.load(open("./data/secrets.json", "r"))["tmap"]["appkey"],
    "city_do": quote("서울특별시"),
    "gu_gun": quote("강남구"),
    "dong": quote("청담동"),
    "bunji": quote("41-2"),
    "detailAddress": quote("강남구청역 4번 출구"),
    "addressFlag": "F01"
}

res = requests.get(url, params=payload, headers=headers)
json.loads(res.text)

{'coordinateInfo': {'coordType': 'WGS84GEO',
  'addressFlag': 'F01',
  'matchFlag': 'M12',
  'lat': '37.519357',
  'lon': '127.041283',
  'city_do': '서울',
  'gu_gun': '강남구',
  'eup_myun': '',
  'legalDong': '청담동',
  'legalDongCode': '1168010400',
  'adminDong': '청담동',
  'adminDongCode': '1168056500',
  'ri': '',
  'bunji': '40-8',
  'newMatchFlag': '',
  'newLat': '',
  'newLon': '',
  'newRoadName': '',
  'newBuildingIndex': '',
  'newBuildingName': '',
  'newBuildingCateName': '',
  'remainder': '강남구청역 4번 출구'}}

In [80]:
url = "https://apis.openapi.sk.com/tmap/pois?version=1"

# headers = {"appKey": json.load(open("./data/secrets.json", "r"))["tmap"]["appkey"]}

payload = {
    "appKey": json.load(open("./data/secrets.json", "r"))["tmap"]["appkey"],
    "searchType": "all",
    "searchKeyword": parse.quote("역삼역"),
    "areaLLCode": "11",
    "areaLMCode": "680"
}

res = requests.get(url, params=payload)
res
# json.loads(res.text)

<Response [204]>

In [69]:
parse.quote("강남구청역 4번출구")

# 11 680 10100
# 11
# 680
# 10400

'%EA%B0%95%EB%82%A8%EA%B5%AC%EC%B2%AD%EC%97%AD%204%EB%B2%88%EC%B6%9C%EA%B5%AC'

# 전동킥보드 전용 주차구역

In [91]:
park_raw = pd.read_csv(raw_path + "서울특별시 강남구_전동킥보드 주차구역_20221031.csv", encoding="cp949")

col_list = ["Name", "Latitude", "Longitude"]
park = pd.DataFrame(columns=col_list)

for address in park_raw["도로명 주소"]:
    lat, lng, addr = get_location(address)
    park = pd.concat([park, pd.DataFrame([[addr, lat, lng]], columns=col_list)], ignore_index=True)

park = park.reset_index()
park["type"] = "kickboard"

park.head()
park.to_csv(temp_path + "kickboard.csv", index=False, encoding="utf-8")

In [92]:
park

,index,Name,Latitude,Longitude,type
0,0,서울 강남구 청담동 41-2,37.517641,127.041673,kickboard
1,1,서울 강남구 삼성동 145,37.512883,127.053852,kickboard
2,2,서울 강남구 역삼동 717,37.500605,127.038096,kickboard
3,3,서울 강남구 삼성동 52,37.518540,127.050709,kickboard
4,4,서울 강남구 논현동 129-1,37.513568,127.030541,kickboard
5,5,서울 강남구 신사동 501-2,37.516780,127.020108,kickboard


# 자전거 주차시설

In [184]:
bicycle = pd.read_csv(raw_path + "서울시 자전거 편의시설.csv", encoding="cp949")
bicycle[["y 좌표", "x 좌표"]]
bicycle

,순서,시설ID,컨텐츠 명,사용유무,테마 타입,x 좌표,y 좌표,좌표 배열,좌표타입,메인 이미지,...,상세정보 이름 6,상세정보 값 6,상세정보 이름 7,상세정보 값 7,상세정보 이름 8,상세정보 값 8,상세정보 이름 9,상세정보 값 9,상세정보 이름 10,상세정보 값 10
0,1,공덕전철역 2번출구,자전거 일반거치대,사용,일반,126.950520,37.544413,[],Point,NaN,...,운영형태(*수리센터만 선택하세요.),NaN,수리범위(*수리센터만 선택하고 예 경정비 무상 부품교체 실비 등.),NaN,이동수리센터 순회일정(*이동수리센터만 입력 예) 자치구 순회일정 안내 게시판 링크),NaN,유의사항(예 O0학교 운영시간 외 이용불가 ),NaN,마포구청 교통행정과,마포구청 교통행정과
1,2,동대문구_자전거보관대_120,_자전거보관대,사용,일반,127.066702,37.588619,[],Point,NaN,...,운영형태(*수리센터만 선택하세요.),NaN,수리범위(*수리센터만 선택하고 예 경정비 무상 부품교체 실비 등.),NaN,이동수리센터 순회일정(*이동수리센터만 입력 예) 자치구 순회일정 안내 게시판 링크),NaN,유의사항(예 O0학교 운영시간 외 이용불가 ),NaN,NaN,NaN
2,3,강동구_수리센터_001,강동구자전거서비스센터,사용,일반,127.156898,37.555645,[],Point,NaN,...,운영형태(*수리센터만 선택하세요.),상설|,공임비 무상 부품교체 실비,NaN,이동수리센터 순회일정(*이동수리센터만 입력 예) 자치구 순회일정 안내 게시판 링크),NaN,유의사항(예 O0학교 운영시간 외 이용불가 ),NaN,NaN,NaN
3,4,한강_수리센터_003,강서 자전거 대여소 수리센터,사용,일반,126.821933,37.584380,[],Point,NaN,...,운영형태(*수리센터만 선택하세요.),NaN,수리범위(*수리센터만 선택하고 예 경정비 무상 부품교체 실비 등.),NaN,이동수리센터 순회일정(*이동수리센터만 입력 예) 자치구 순회일정 안내 게시판 링크),NaN,유의사항(예 O0학교 운영시간 외 이용불가 ),NaN,NaN,NaN
4,5,구로구_자전거주차장_002,개봉역 자전거주차장,사용,일반,126.859377,37.495407,[126.8593768982035537.49540727677393],Point,NaN,...,운영형태(*수리센터만 선택하세요.),NaN,수리범위(*수리센터만 선택하고 예 경정비 무상 부품교체 실비 등.),NaN,이동수리센터 순회일정(*이동수리센터만 입력 예) 자치구 순회일정 안내 게시판 링크),NaN,유의사항(예 O0학교 운영시간 외 이용불가 ),NaN,구로구 교통행정과,구로구 교통행정과
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2518,2548,합정역 자전거주차장,합정역 자전거주차장,사용,일반,126.912560,37.549623,[],Point,NaN,...,운영형태(*수리센터만 선택하세요.),NaN,수리범위(*수리센터만 선택하고 예 경정비 무상 부품교체 실비 등.),NaN,이동수리센터 순회일정(*이동수리센터만 입력 예) 자치구 순회일정 안내 게시판 링크),NaN,유의사항(예 O0학교 운영시간 외 이용불가 ),NaN,NaN,NaN
2519,2549,노원구_공기주입기_022,호산나교회 옆_공기주입기,사용,일반,127.060355,37.639370,[],Point,https://map.seoul.go.kr/smgis/ucimgs/conts/111...,...,운영형태(*수리센터만 선택하세요.),NaN,수리범위(*수리센터만 선택하고 예 경정비 무상 부품교체 실비 등.),NaN,이동수리센터 순회일정(*이동수리센터만 입력 예) 자치구 순회일정 안내 게시판 링크),NaN,유의사항(예 O0학교 운영시간 외 이용불가 ),NaN,노원구청 교통지도과,노원구청 교통지도과
2520,2550,홍대입구역 자전거주차장,홍대입구역 자전거주차장,사용,일반,126.929124,37.556402,[],Point,NaN,...,운영형태(*수리센터만 선택하세요.),NaN,수리범위(*수리센터만 선택하고 예 경정비 무상 부품교체 실비 등.),NaN,이동수리센터 순회일정(*이동수리센터만 입력 예) 자치구 순회일정 안내 게시판 링크),NaN,유의사항(예 O0학교 운영시간 외 이용불가 ),NaN,마포구청 교통행정과,마포구청 교통행정과
2521,2551,중랑구_거치대_001,화랑대역 주차장 내,사용,일반,127.086062,37.619934,[127.086061916025237.61993353943017],Point,NaN,...,운영형태(*수리센터만 선택하세요.),NaN,수리범위(*수리센터만 선택하고 예 경정비 무상 부품교체 실비 등.),NaN,이동수리센터 순회일정(*이동수리센터만 입력 예) 자치구 순회일정 안내 게시판 링크),NaN,유의사항(예 O0학교 운영시간 외 이용불가 ),NaN,NaN,NaN


In [189]:
bicycle["상세정보 값 10"].unique()

array(['마포구청 교통행정과', nan, '구로구 교통행정과', '노원구청 교통지도과', '영등포구 교통행정과',
       '한강사업본부 시민활동지원과', '노원구 교통지도과', '서울교통공사 9호선운영부문',
       '마포구청 교통행정과(02-3153-9643)', '서초구청 교통행정과', '광진구청 교통행정과',
       '성북지역자활센터 02-927-2420', '강남구청 교통행정과', '성동구청 교통행정과', '성북지역자활센터 ',
       '구로구청 교통행정과', '서울구로삶터지역자활센터', '영등포구청 교통행정과',
       '서울중랑유린지역자활센터 지역재생사업단', '양천구청 교통행정과', '강서구청 교통시설팀', '교통행정과 교통시설팀',
       '양천구 교통행정과', '강서구청 교통행정과 교통시설팀', '교통시설팀 교통행정과', '용산구청 교통행정과',
       '푸른도시과 관리', '장충파출소', '관리부서: 강서구청 교통행정과', '성북구청 교통행정과', '성북구 교통행정과',
       '교통행정과', '은평구 교통행정과'], dtype=object)

In [182]:
bicycle = bicycle[(bicycle["상세정보 이름 5"]!="설치/이용가능대수") & (bicycle["상세정보 이름 5"]!="설치/이용가능대수(예 00대)")][["시설ID", "컨텐츠 명", "y 좌표", "x 좌표"]]
bicycle = bicycle.rename({"y 좌표": "Latitude", "x 좌표": "Longitude"}, axis=1).reset_index(drop=True)

for i in tqdm(range(len(bicycle))):
    bicycle.loc[i, "gangnam"] = whether_gangnam(bicycle.loc[i, "Latitude"], bicycle.loc[i, "Longitude"])

100%|████████████████████████████████████████████████████████████████████████████████| 365/365 [01:11<00:00,  5.12it/s]


In [178]:
bicycle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   시설ID       365 non-null    object 
 1   컨텐츠 명      365 non-null    object 
 2   Latitude   365 non-null    float64
 3   Longitude  365 non-null    float64
dtypes: float64(2), object(2)
memory usage: 11.5+ KB


In [174]:
bicycle = bicycle[bicycle["gangnam"]==True]
# bus = bicycle.drop(["gangnam"], axis=1).reset_index(drop=True).reset_index()

print(len(bicycle))
print(bicycle.info())
bicycle.head()

0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   시설ID       0 non-null      object 
 1   컨텐츠 명      0 non-null      object 
 2   Latitude   0 non-null      float64
 3   Longitude  0 non-null      float64
 4   gangnam    0 non-null      object 
dtypes: float64(2), object(3)
memory usage: 0.0+ bytes
None


,시설ID,컨텐츠 명,Latitude,Longitude,gangnam


# concat

In [ ]:
data_list = ["kickboard", "bicycle"]

In [ ]:
parkingzone = pd.DataFrame(columns=["index", "Name", "Latitude", "Longitude", "type"])

for data in data_list:
    data_df = pd.read_csv(f"{temp_path}{data}.csv")
    data_df["type"] = data
    parkingzone = pd.concat([parkingzone, data_df], ignore_index=True)

print(len(parkingzone))
print(parkingzone.info())
parkingzone.head()

In [190]:
kickboard = pd.read_csv(temp_path + "kickboard.csv")
kickboard

,index,Name,Latitude,Longitude,type
0,0,서울 강남구 청담동 41-2,37.517641,127.041673,kickboard
1,1,서울 강남구 삼성동 145,37.512883,127.053852,kickboard
2,2,서울 강남구 역삼동 717,37.500605,127.038096,kickboard
3,3,서울 강남구 삼성동 52,37.518540,127.050709,kickboard
4,4,서울 강남구 논현동 129-1,37.513568,127.030541,kickboard
5,5,서울 강남구 신사동 501-2,37.516780,127.020108,kickboard


In [191]:
def get_dong(latitude, longitude):
    url = "https://apis.openapi.sk.com/tmap/geo/reversegeocoding?version=1"

    payload = {
        "appKey": "l7xx0ffb87c22bdb45ae8facaeeb7958ad36",
        "lon": longitude,
        "lat": latitude,
    }

    res = requests.get(url, params=payload)

    try:
        dongCode = json.loads(res.text)["addressInfo"]["legalDongCode"][5:8]
    except:
        return "ERROR"

    if dongCode in dong["dongCode"].unique():
        return dongCode
    else:
        return "Invalid Code"

In [195]:
for i in tqdm(range(len(kickboard))):
    kickboard.loc[i, "dongCode"] = get_dong(kickboard.loc[i, "Latitude"], kickboard.loc[i, "Longitude"])

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  7.38it/s]


In [193]:
dong = pd.read_csv(data_path + "dong.csv", dtype="object")

In [197]:
kickboard.to_csv(data_path + "parkingzone.csv", index=False, encoding="utf-8")